In [ ]:
from pathlib import Path
import matplotlib as mpl

import matplotlib.pyplot as plt
import colorcet  # noqa: F401
from flow_analysis_comps.video_manipulation.control_class import videoControl
from flow_analysis_comps.Classic.extract_velocity import kymoAnalyser, videoDeltas
from flow_analysis_comps.Fourier.OrientationSpaceManager import orientationSpaceManager

plt.close("all")

%matplotlib widget

%load_ext autoreload
%autoreload 2

In [ ]:
video_root_folder = Path(
    r'/Users/simonvanstaalduine/AMOLF-SHIMIZU Dropbox/Simon van Staalduine/033'
)
info_file_address = Path(video_root_folder / "videoInfo.txt")
video_operator = videoControl(video_root_folder, info_file_address, resolution=1)
mean_img = video_operator.mean_img
# video_operator.save_mp4_video()

In [ ]:
kymographs = video_operator.get_kymographs()
# video_operator.save_edge_videos(video_root_folder)

In [ ]:
import numpy as np


# video_deltas = videoDeltas(
#     delta_x=video_operator.space_pixel_size,
#     delta_t=video_operator.time_pixel_size
# )

video_deltas = videoDeltas(
    delta_t=1,
    delta_x=1
)

OSFilter = orientationSpaceManager(
    0.15,
    freq_width=0.2,
    K=16,
    x_spacing=video_deltas.delta_x,
    y_spacing=video_deltas.delta_t,
)

new_order = 8
speed_limit = 10

for key, kymo in kymographs.items():
    analyser = kymoAnalyser(kymo, video_deltas=video_deltas, speed_threshold=speed_limit)
    analyser.plot_kymo_fields()
    # fig, ax = analyser.plot_summary()
    # fig.savefig(video_root_folder / f"{key}_summary.png")
    image_l, image_r = analyser.kymograph_decomposed_directions
    invert_im = video_operator.video_info.mode == "brightfield"
    # _, _, histo_l = OSFilter.demo_image(
    #     image_l,
    #     video_operator.space_pixel_size,
    #     video_operator.time_pixel_size,
    #     order=new_order,
    #     thresh_method="triangle",
    #     invert=invert_im,
    #     histo_thresh=1.2,
    #     speed_extent=speed_limit,
    # )
    # _, _, histo_r = OSFilter.demo_image(
    #     image_r,
    #     video_operator.space_pixel_size,
    #     video_operator.time_pixel_size,
    #     order=new_order,
    #     thresh_method="triangle",
    #     invert=invert_im,
    #     histo_thresh=1.2,
    #     speed_extent=speed_limit,
    # )
    _, _, histo_b = OSFilter.demo_image(
        kymo,
        video_deltas.delta_x,
        video_deltas.delta_t,
        order=new_order,
        thresh_method="triangle",
        invert=invert_im,
        histo_thresh=.7,
        speed_extent=speed_limit,
    )

In [ ]:
fig, ax = plt.subplots()
histo_l_n = histo_l / np.max(histo_l)
histo_r_n = histo_r / np.max(histo_r)
ax.imshow((histo_l_n + histo_r_n).T,
            cmap="cet_CET_L16",
            extent=(0, len(histo_l) * video_deltas.delta_t, -speed_limit, speed_limit),
            aspect='auto')